In [12]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os 
import csv
import psycopg2 
import json
from typing import TypedDict
load_dotenv()

URL_DB = os.environ.get("URL_DB")
CONN = psycopg2.connect(URL_DB)
CURSOR = CONN.cursor()

In [13]:
# NOTE: GROUP LOCATION BY COUNTRIES. 
# NOTE: WHERE IS LOCATION == NONE?
# NOTE: WHERE IS DESCRIPTION == NONE?


## TODO: NEED TO TRANSFROM COMMON TWO WORDS SUCH AS UNITED STATES INTO "USA"; "NEW YORK", 

timestamp = "2024-05-01 00:00:00.000000"

CURSOR.execute(
	f"SELECT location FROM main_jobs  WHERE timestamp > '{timestamp}'"
)
new_data = CURSOR.fetchall()
df = pd.DataFrame(new_data)

df.rename(columns={0: "location"}, inplace=True)



In [14]:
def load_json_file(file_path: str):
	with open(file_path, 'r') as file:
		return json.load(file)

def save_json_file(data: dict, file_path: str) -> None:
	with open(file_path, 'w') as file:
		json.dump(data, file, indent=4)

In [15]:


def transform_data(input_data: dict) -> dict[str, dict[str, list[dict[str, list[str]]]]]:
	"""Simplify the data structure of 'all_locations.json' for better and easier mapping"""
	result = {}
	for continent, data in input_data.items():
		result[continent] = {"Countries": []}
		for country in data["Countries"]:
			country_name: str = country["country_name"].upper()
			country_code = country["country_code"].upper()
			if country["capital_english"] != "NaN":
				capital_english = country["capital_english"].upper()
			
			subdivisions = []
			if isinstance(country["subdivisions"], list):
				subdivisions = [sub["subdivisions_name"].upper() for sub in country["subdivisions"]]
			elif country["subdivisions"] != "NaN":
				subdivisions = [country["subdivisions"].upper()]
			
			transformed_country = {
				country_name: [country_code, capital_english] + subdivisions
			}
			result[continent]["Countries"].append(transformed_country)
	
	return result

#Example usage:

"""
input_file = "/root/JobsCrawler/src/notebooks/all_locations.json"
output_file = "/root/JobsCrawler/src/notebooks/all_locations_transformed.json"

input_data = load_json_file(input_file)
transformed_data = transform_data(input_data)
save_json_file(transformed_data, output_file)

print("Data transformation complete. Result saved to", output_file)
"""

'\ninput_file = "/root/JobsCrawler/src/notebooks/all_locations.json"\noutput_file = "/root/JobsCrawler/src/notebooks/all_locations_transformed.json"\n\ninput_data = load_json_file(input_file)\ntransformed_data = transform_data(input_data)\nsave_json_file(transformed_data, output_file)\n\nprint("Data transformation complete. Result saved to", output_file)\n'

In [16]:

df['original_index'] = df.index

df["location"] = df["location"].str.replace(",", "", regex=False).str.replace(")", "", regex=False).str.replace("(", "", regex=False).str.replace("|", " ", regex=False)

df["location"] = df["location"].str.strip().str.split()
df = df.explode("location").reset_index(drop=True)

df.head(25)

df.to_csv("/root/JobsCrawler/src/notebooks/all_location_words.csv")


In [17]:
# Get all unique words
unique_words = df["location"].unique()

# Create a new DataFrame with these unique words
unique_words_df = pd.DataFrame({"unique_word": unique_words})


unique_words_df.to_csv("/root/JobsCrawler/src/notebooks/unique_words.csv")



## naive approach

In [18]:


## TODO: AS SOME LOCATIONS CONTAIN MORE THAN A WORD, IT SHOULD ADD THE NEXT WORD IF THE RESULT IS UNKNOWN, AND IF THEIR INDEX ARE THE SAME. ELSE FINALLY ASSING UNKNOWN AGAIN. MAKE A FUNCTION THAT CONCATENATES THE WORD THAT COULD NOT BE FOUND IN LOC.LOWER AND ADD ITS NEXT WORD ONLY IF THEIR INDEX ARE EQUAL. FOR EXAMPLE, THIS FUNCTION WOULD NOT MATCH THE WORD "SAN" BUT IF WE DO THIS, THEN THE CONCATENATED STRING WOULD BE "SAN FRANCISCO", WHICH SHOULD FIND ITS MATCH.

"""

def add_location_tags(df: pd.DataFrame, json_file_path: str) -> pd.DataFrame:
	location_data = load_json_file(json_file_path)
	def find_location_tag(word: str) -> str | float:
			word_str = str(word)
			for continent, countries in location_data.items():
				for country in countries['Countries']:
					for country_name, locations in country.items():
						for loc in locations:
							if word_str.lower() in loc.lower():
								return country_name
						if word_str.lower() == country_name.lower():
							return country_name
				if word_str.lower() == continent.lower():
					return continent
			return np.nan
	df['location_tag'] = df[0].apply(find_location_tag)
	return df

json_file_path = '/root/JobsCrawler/src/notebooks/all_locations_transformed.json'
result2_df = add_location_tags(df, json_file_path)
print(result2_df)

result2_df.to_csv("/root/JobsCrawler/src/notebooks/country_mapping.csv")

nan_count_per_column = result2_df.isna().sum()

print("nan_count_per_column", nan_count_per_column)

"""

'\n\ndef add_location_tags(df: pd.DataFrame, json_file_path: str) -> pd.DataFrame:\n\tlocation_data = load_json_file(json_file_path)\n\tdef find_location_tag(word: str) -> str | float:\n\t\t\tword_str = str(word)\n\t\t\tfor continent, countries in location_data.items():\n\t\t\t\tfor country in countries[\'Countries\']:\n\t\t\t\t\tfor country_name, locations in country.items():\n\t\t\t\t\t\tfor loc in locations:\n\t\t\t\t\t\t\tif word_str.lower() in loc.lower():\n\t\t\t\t\t\t\t\treturn country_name\n\t\t\t\t\t\tif word_str.lower() == country_name.lower():\n\t\t\t\t\t\t\treturn country_name\n\t\t\t\tif word_str.lower() == continent.lower():\n\t\t\t\t\treturn continent\n\t\t\treturn np.nan\n\tdf[\'location_tag\'] = df[0].apply(find_location_tag)\n\treturn df\n\njson_file_path = \'/root/JobsCrawler/src/notebooks/all_locations_transformed.json\'\nresult2_df = add_location_tags(df, json_file_path)\nprint(result2_df)\n\nresult2_df.to_csv("/root/JobsCrawler/src/notebooks/country_mapping.csv")\

## sliding window approach

In [19]:

class Countries(TypedDict):
	country_name: str
	locations: list[str]

class WorldLocations(TypedDict):
	continent: str
	areas: list[str]
	countries: list[Countries]




def find_location_tag(word: str, location_data: WorldLocations) -> str:
	word_upper = word.upper()
	for continent, countries in location_data.items():
		if word_upper == continent.upper():
			return continent
		for zone in countries['Zones']:
			if word_upper == zone:
				return continent
		for country in countries['Countries']:
			for country_name, locations in country.items():
				if word_upper == country_name or word_upper in [loc for loc in locations]:
					return country_name
	return ""

def add_location_tags(df: pd.DataFrame, json_file_path: str) -> pd.DataFrame:
	location_data = load_json_file(json_file_path)
	result = []
	i = 0
	while i < len(df):
		current_word = str(df.iloc[i, 0])
		current_original_index = df.loc[i, "original_index"]
		
		tag = find_location_tag(current_word, location_data)
		
		if tag:
			result.append(tag)
			i += 1
		else:
			# If no match, try to concatenate with the next word if it has the same original_index
			if i + 1 < len(df) and df.loc[i + 1, "original_index"] == current_original_index:
				next_word = str(df.iloc[i + 1, 0])
				compound_word = f"{current_word} {next_word}"
				tag = find_location_tag(compound_word, location_data)
				
				if tag:
					result.extend([tag, tag])
					i += 2
				else:
					result.append(np.nan)
					i += 1
			else:
				result.append(np.nan)
				i += 1

	df['location_tag'] = result
	return df

json_file_path = '/root/JobsCrawler/src/notebooks/all_locations_transformed.json'
result_df = add_location_tags(df, json_file_path)
print(result_df)

result_df.to_csv("/root/JobsCrawler/src/notebooks/country_mapping1.csv")


      location  original_index location_tag
0       Canada               0       CANADA
1       Canada               1       CANADA
2       Remote               2          NaN
3       Hybrid               2          NaN
4       Remote               2          NaN
...        ...             ...          ...
56840   Remote           11816          NaN
56841    India           11817        INDIA
56842  Chennai           11817          NaN
56843    India           11817        INDIA
56844   Remote           11817          NaN

[56845 rows x 3 columns]


In [20]:

nan_count_per_column = result_df.isna().sum()

print(nan_count_per_column)


location            129
original_index        0
location_tag      17630
dtype: int64


## do the same but for the unique words. To see all the words that are not currently mapped

In [21]:

def simple_add_location_tags(df: pd.DataFrame, json_file_path: str) -> pd.DataFrame:
	location_data = load_json_file(json_file_path)
	result = []
	i = 0
	while i < len(df):
		current_word = str(df.iloc[i, 0])
		
		tag = find_location_tag(current_word, location_data)
		
		if tag:
			result.append(tag)
		else:
			result.append(np.nan)
		
		i += 1

	df['location_tag'] = result
	return df

json_file_path = '/root/JobsCrawler/src/notebooks/all_locations_transformed.json'


unique_mapped_words_df = simple_add_location_tags(unique_words_df, json_file_path)

unique_mapped_words_df.to_csv("/root/JobsCrawler/src/notebooks/unique_word_mapping.csv")


unique_mapped_words_df


,unique_word,location_tag
0,Canada,CANADA
1,Remote,NaN
2,Hybrid,NaN
3,USA,UNITED STATES
4,Europe,Europe
...,...,...
1043,Cencora,NaN
1044,Tunis,TUNISIA
1045,Tacoma,NaN
1046,EUR3500,NaN


In [22]:
unique_mapped_words_df


,unique_word,location_tag
0,Canada,CANADA
1,Remote,NaN
2,Hybrid,NaN
3,USA,UNITED STATES
4,Europe,Europe
...,...,...
1043,Cencora,NaN
1044,Tunis,TUNISIA
1045,Tacoma,NaN
1046,EUR3500,NaN


In [23]:
nan_count_per_column = unique_mapped_words_df.isna().sum()

print(nan_count_per_column)

unique_word       1
location_tag    691
dtype: int64
